Import the necessary libraries and modules.

In [58]:
import pandas as pd
import datetime as dt
import csv
import math

Change the Path variable to point to your CSV file

In [59]:
# C:\Users\joacosta\Work\Beumer\NortFord\Data Analitics\Programs to extract Data\Log Monitor Files\09-18-2025\iteminducted.csv

In [60]:
# path = r"C:\Users\joacosta\Dev\Python\ORF5\Data\iu006rate.csv"
path = r"C:\Users\joacosta\Dev\Python\ORF5\Data\iu007_VS_.csv"

In [61]:
#  Raw read: semicolon-delimited, no header, don't treat quotes specially
df = pd.read_csv(
    path,
    sep=";",
    header=None,
    engine="python",
    quoting=csv.QUOTE_NONE,      # make " just a character
    skipinitialspace=True,
    on_bad_lines="skip",         # skip malformed rows if any
    dtype=str                    # keep everything as text for now
)

# Trim spaces/quotes around each field
df = df.apply(lambda s: s.str.strip().str.strip('"'))

names = [
    "stamp_raw","flag","system","hostport","module","unused1","unused2","run_hms",
    "seq","mc","code","sq","event","awcsAction","plcRecordNo","itemID","indexNo",
    "awcsStateNow","awcsStateNew","inductionStatus","inductionNo","destinationNo",
    "carrierNo","carrierCount","comHost","comMode","telegramType"
]

df.columns = names[:df.shape[1]]
df.head(10)


,stamp_raw,flag,system,hostport,module,unused1,unused2,run_hms,seq,mc,code,sq
0,250918 140653 702,N,SMC,10.158.244.100:7200,AWCS.Comm,,,"01:48:17,593",MC01,54123,SQ 100,"->{event: ""AwcsConverterReceive40ItemInducted""..."
1,250918 140654 205,N,SMC,10.158.244.100:7200,AWCS.Comm,,,"01:48:18,111",MC01,54123,SQ 133,"->{event: ""AwcsConverterReceive40ItemInducted""..."
2,250918 140655 238,N,SMC,10.158.244.100:7200,AWCS.Comm,,,"01:48:19,116",MC01,54123,SQ 189,"->{event: ""AwcsConverterReceive40ItemInducted""..."
3,250918 140657 041,N,SMC,10.158.244.100:7200,AWCS.Comm,,,"01:48:20,948",MC01,54123,SQ 049,"->{event: ""AwcsConverterReceive40ItemInducted""..."
4,250918 140658 240,N,SMC,10.158.244.100:7200,AWCS.Comm,,,"01:48:22,147",MC01,54123,SQ 119,"->{event: ""AwcsConverterReceive40ItemInducted""..."
5,250918 140659 665,N,SMC,10.158.244.100:7200,AWCS.Comm,,,"01:48:23,540",MC01,54123,SQ 215,"->{event: ""AwcsConverterReceive40ItemInducted""..."
6,250918 140700 896,N,SMC,10.158.244.100:7200,AWCS.Comm,,,"01:48:24,797",MC01,54123,SQ 027,"->{event: ""AwcsConverterReceive40ItemInducted""..."
7,250918 140702 506,N,SMC,10.158.244.100:7200,AWCS.Comm,,,"01:48:26,392",MC01,54123,SQ 112,"->{event: ""AwcsConverterReceive40ItemInducted""..."
8,250918 140703 069,N,SMC,10.158.244.100:7200,AWCS.Comm,,,"01:48:26,938",MC01,54123,SQ 145,"->{event: ""AwcsConverterReceive40ItemInducted""..."
9,250918 140703 989,N,SMC,10.158.244.100:7200,AWCS.Comm,,,"01:48:27,897",MC01,54123,SQ 204,"->{event: ""AwcsConverterReceive40ItemInducted""..."


Adding timestamp Column in a legible format

In [62]:
sr = df["stamp_raw"].str.replace(" ", "", regex=False)  # e.g. 250917160001278
df["timestamp"] = pd.to_datetime(sr, format="%y%m%d%H%M%S%f", errors="coerce")

df.head()


,stamp_raw,flag,system,hostport,module,unused1,unused2,run_hms,seq,mc,code,sq,timestamp
0,250918 140653 702,N,SMC,10.158.244.100:7200,AWCS.Comm,,,"01:48:17,593",MC01,54123,SQ 100,"->{event: ""AwcsConverterReceive40ItemInducted""...",2025-09-18 14:06:53.702
1,250918 140654 205,N,SMC,10.158.244.100:7200,AWCS.Comm,,,"01:48:18,111",MC01,54123,SQ 133,"->{event: ""AwcsConverterReceive40ItemInducted""...",2025-09-18 14:06:54.205
2,250918 140655 238,N,SMC,10.158.244.100:7200,AWCS.Comm,,,"01:48:19,116",MC01,54123,SQ 189,"->{event: ""AwcsConverterReceive40ItemInducted""...",2025-09-18 14:06:55.238
3,250918 140657 041,N,SMC,10.158.244.100:7200,AWCS.Comm,,,"01:48:20,948",MC01,54123,SQ 049,"->{event: ""AwcsConverterReceive40ItemInducted""...",2025-09-18 14:06:57.041
4,250918 140658 240,N,SMC,10.158.244.100:7200,AWCS.Comm,,,"01:48:22,147",MC01,54123,SQ 119,"->{event: ""AwcsConverterReceive40ItemInducted""...",2025-09-18 14:06:58.240


Getting the Start time and End time of the data and calculating the delta of time plus how many records we have in this dataset

In [63]:
valid = df["timestamp"].notna()
start_time = df.loc[valid, "timestamp"].min()
end_time   = df.loc[valid, "timestamp"].max()
general_delta_time = end_time - start_time

print(f"Start Time of Data : {start_time}")
print(f"End Time of Data  : {end_time}")
print(f"Delta of Time : {general_delta_time}  (~{general_delta_time.total_seconds()/60:.2f} minutes)")
print(f"Rows in the Dataset : {valid.sum()} valid / {len(df)-valid.sum()} invalid")

Start Time of Data : 2025-09-18 14:06:53.702000
End Time of Data  : 2025-09-18 14:42:09.330000
Delta of Time : 0 days 00:35:15.628000  (~35.26 minutes)
Rows in the Dataset : 912 valid / 0 invalid


In [64]:
# # data limits from your parsed timestamps
# data_min = df["timestamp"].min()
# data_max = df["timestamp"].max()

# def select_window(df, start_input, minutes=10):
#     base_date = data_min.normalize()                # date of your dataset
#     ts_in = pd.to_datetime(start_input)             # e.g. "16:40:00" or "4:40 PM"
#     start = base_date + (ts_in - ts_in.normalize()) # anchor time-of-day to base_date
#     end   = start + pd.Timedelta(minutes=minutes)

#     print(f"Data range : {data_min} → {data_max}")
#     print(f"Requested  : {start} → {end}  ({minutes} min)")
#     print("Start OK   :", data_min <= start <= data_max)
#     if end > data_max:
#         print("Note       : End is beyond data_max; results will stop at data_max.")

#     mask = (df["timestamp"] >= start) & (df["timestamp"] <= end)
#     return df.loc[mask].copy(), start, end

# # EXAMPLE:
# window_df, start_ts, end_ts = select_window(df, "16:06:00", minutes=10)
# window_df.head(10)

Asking to the user what time window wants to analyze.

In general we aim for the best 10 minutes in the production state.

In [65]:
data_min = df["timestamp"].min()
base_date = data_min.normalize()

def select_window_cli_24h(df):
    s = input("Start time 24h (e.g. 16:00 or 16:00:30 or 16): ").strip()
    m = input("Window minutes [10]: ").strip()
    minutes = int(m) if m else 10
    try:
        t = dt.time.fromisoformat(s)
    except ValueError:
        t = dt.time(int(s), 0, 0)
    start = pd.Timestamp(dt.datetime.combine(base_date.date(), t))
    end   = start + pd.Timedelta(minutes=minutes)
    mask  = (df["timestamp"] >= start) & (df["timestamp"] <= end)
    win   = df.loc[mask].copy()
    print(f"Window: {start} → {end}  ({minutes} min) | Rows: {len(win)}")
    return win, start, end

window_df, start_ts, end_ts = select_window_cli_24h(df)
window_df.head(10)


Window: 2025-09-18 14:10:00 → 2025-09-18 14:20:00  (10 min) | Rows: 398


,stamp_raw,flag,system,hostport,module,unused1,unused2,run_hms,seq,mc,code,sq,timestamp
128,250918 141000 401,N,SMC,10.158.244.100:7200,AWCS.Comm,,,"01:51:24,306",MC01,54123,SQ 211,"->{event: ""AwcsConverterReceive40ItemInducted""...",2025-09-18 14:10:00.401
129,250918 141002 788,N,SMC,10.158.244.100:7200,AWCS.Comm,,,"01:51:26,687",MC01,54123,SQ 061,"->{event: ""AwcsConverterReceive40ItemInducted""...",2025-09-18 14:10:02.788
130,250918 141004 784,N,SMC,10.158.244.100:7200,AWCS.Comm,,,"01:51:28,648",MC01,54123,SQ 162,"->{event: ""AwcsConverterReceive40ItemInducted""...",2025-09-18 14:10:04.784
131,250918 141005 703,N,SMC,10.158.244.100:7200,AWCS.Comm,,,"01:51:29,608",MC01,54123,SQ 212,"->{event: ""AwcsConverterReceive40ItemInducted""...",2025-09-18 14:10:05.703
132,250918 141006 866,N,SMC,10.158.244.100:7200,AWCS.Comm,,,"01:51:30,772",MC01,54123,SQ 012,"->{event: ""AwcsConverterReceive40ItemInducted""...",2025-09-18 14:10:06.866
133,250918 141008 305,N,SMC,10.158.244.100:7200,AWCS.Comm,,,"01:51:32,211",MC01,54123,SQ 101,"->{event: ""AwcsConverterReceive40ItemInducted""...",2025-09-18 14:10:08.305
134,250918 141009 529,N,SMC,10.158.244.100:7200,AWCS.Comm,,,"01:51:33,427",MC01,54123,SQ 168,"->{event: ""AwcsConverterReceive40ItemInducted""...",2025-09-18 14:10:09.529
135,250918 141010 545,N,SMC,10.158.244.100:7200,AWCS.Comm,,,"01:51:34,429",MC01,54123,SQ 236,"->{event: ""AwcsConverterReceive40ItemInducted""...",2025-09-18 14:10:10.545
136,250918 141012 241,N,SMC,10.158.244.100:7200,AWCS.Comm,,,"01:51:36,122",MC01,54123,SQ 085,"->{event: ""AwcsConverterReceive40ItemInducted""...",2025-09-18 14:10:12.241
137,250918 141012 992,N,SMC,10.158.244.100:7200,AWCS.Comm,,,"01:51:36,892",MC01,54123,SQ 131,"->{event: ""AwcsConverterReceive40ItemInducted""...",2025-09-18 14:10:12.992


Calculating metrics as Items per second, Average time between items, Average time per item

In [66]:
n_items = len(window_df)
delta_secs = general_delta_time.total_seconds()
delta_min = delta_secs / 60.0

sec_per_bag   = (delta_secs / n_items) if n_items > 0 else float("nan")
items_per_sec = (n_items / delta_secs) if delta_secs > 0 else float("nan")
items_per_min = items_per_sec * 60 if delta_secs > 0 else float("nan")

print(f"Items in window_df : {n_items}")
print(f"Overall delta (s)  : {delta_secs:.2f}")
print(f"Overall delta (min): {delta_min:.2f}")
print(f"Sec per bag        : {sec_per_bag:.3f} s/bag")
print(f"Throughput         : {items_per_sec:.4f} items/s  ({items_per_min:.2f} items/min)")


Items in window_df : 398
Overall delta (s)  : 2115.63
Overall delta (min): 35.26
Sec per bag        : 5.316 s/bag
Throughput         : 0.1881 items/s  (11.29 items/min)


In [67]:
TARGET_PPH = 2820  # pieces per hour target

# PPH using overall dataset duration (delta from min→max)
pph_overall = n_items * 3600 / delta_secs if delta_secs > 0 else float("nan")

# PPH using the selected window duration
window_secs = (end_ts - start_ts).total_seconds()
pph_window = n_items * 3600 / window_secs if window_secs > 0 else float("nan")

print(f"Items in window_df     : {n_items}")
print(f"PPH (overall delta)    : {pph_overall:.2f}")
print(f"PPH (window {window_secs/60:.1f} min): {pph_window:.2f}")
print(f"Target PPH             : {TARGET_PPH}")
print(f"Attainment vs target (overall): {pph_overall/TARGET_PPH*100:.1f}%")
print(f"Attainment vs target (window) : {pph_window/TARGET_PPH*100:.1f}%")


Items in window_df     : 398
PPH (overall delta)    : 677.25
PPH (window 10.0 min): 2388.00
Target PPH             : 2820
Attainment vs target (overall): 24.0%
Attainment vs target (window) : 84.7%


In [ ]:
# Pass/fail (pass if result >= target)
passed = (pph_window >= TARGET_PPH) if window_secs > 0 and n_items > 0 else False

# How many items would be needed to pass in this same window?
needed_items_for_pass = math.ceil(TARGET_PPH * window_secs / 3600) if window_secs > 0 else float("nan")
items_short = max(0, needed_items_for_pass - n_items) if window_secs > 0 else float("nan")

# Seconds per bag (current vs target) — handy context
spb_current = window_secs / n_items if n_items > 0 else float("inf")
spb_target  = 3600 / TARGET_PPH

# Recommendation text
if window_secs > 0 and n_items > 0 and items_short > 0:
    recommendation = (
        f"Need +{items_short} items in this {window_secs/60:.1f} min window "
        f"or reduce sec/bag to ≤ {spb_target:.2f} (current {spb_current:.2f})."
    )
else:
    recommendation = "On target or above. Maintain current rate."

# --- Summary line pieces ---
line_name = "IU006"
measuring_point = "MP1000605"
meridiem = end_ts.strftime('%p')
time_label = f"{start_ts:%H%M}-{end_ts:%H%M}{meridiem}"
summary_line = f"{line_name};Measuring Point: {measuring_point}: Time {time_label}; Total Inducted = {n_items}"


# Print all the results of the analisis
print("----Dataset Info----")
print(f"Start Time of Data : {start_time}")
print(f"End Time of Data  : {end_time}")
print(f"Delta of Time : {general_delta_time}  (~{general_delta_time.total_seconds()/60:.2f} minutes)")
print(f"Rows in the Dataset : {valid.sum()} valid / {len(df)-valid.sum()} invalid")

print("\n---Requested Time Window and Results---")
print(f"Window Requested: {start_ts} → {end_ts}  (10 min) | Rows: {n_items}")

print(f"Items in window_df : {n_items}")
print(f"Overall delta (s)  : {delta_secs:.2f}")
print(f"Overall delta (min): {delta_min:.2f}")
print(f"Sec per bag        : {sec_per_bag:.3f} s/bag")
print(f"Throughput         : {items_per_sec:.4f} items/s  ({items_per_min:.2f} items/min)")

print("\n---PPH Analysis---")
print(f"Target PPH             : {TARGET_PPH}")
print(f"PPH (window {window_secs/60:.1f} min): {pph_window:.2f}")
print(f"PPH (overall delta)    : {pph_overall:.2f}")
print(f"Attainment vs target (overall): {pph_overall/TARGET_PPH*100:.1f}%")
print(f"Attainment vs target (window) : {pph_window/TARGET_PPH*100:.1f}%")
print(f"\nPPH (window): {pph_window:.2f} | Target: {TARGET_PPH} | Passed: {passed}\n")
print(f"Items in window: {n_items} | Items short to pass: {items_short}")
print(f"Recommendation: {recommendation}")
print(summary_line)


----Dataset Info----
Start Time of Data : 2025-09-18 14:06:53.702000
End Time of Data  : 2025-09-18 14:42:09.330000
Delta of Time : 0 days 00:35:15.628000  (~35.26 minutes)
Rows in the Dataset : 912 valid / 0 invalid

---Requested Time Window and Results---
Window Requested: 2025-09-18 14:10:00 → 2025-09-18 14:20:00  (10 min) | Rows: 398
Items in window_df : 398
Overall delta (s)  : 2115.63
Overall delta (min): 35.26
Sec per bag        : 5.316 s/bag
Throughput         : 0.1881 items/s  (11.29 items/min)

---PPH Analysis---
Target PPH             : 2820
PPH (window 10.0 min): 2388.00
PPH (overall delta)    : 677.25
Attainment vs target (overall): 24.0%
Attainment vs target (window) : 84.7%

PPH (window): 2388.00 | Target: 2820 | Passed: False

Items in window: 398 | Items short to pass: 72
Recommendation: Need +72 items in this 10.0 min window or reduce sec/bag to ≤ 1.28 (current 1.51).
IU006;Measuring Point: MP1000605: Time 1410-1420PM; Total Inducted = 398
